# Geodatenanalyse 2


## Termin Big Data 5 - Modul 2

## *Earth Engine*: Analyse und Export

Ca. 20-30 Minuten

## Inhalt

- Datenanalyse in *Google Earth Engine*
- *ee.Features* inspizieren
- Datensätze exportieren
- Zeitreihen exportieren

Beispiele aus diesem [Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

In [1]:
import geemap
import geemap.colormaps as cm
import datetime as dt
import pandas as pd

In [2]:
import ee
# initialize the connection to the server
ee.Initialize()

## Datenanalyse in *Google EE*

- *EE* kann als Cloud-Computer zur Berechnung von Datensätzen verwendet werden

- Als Beispiel verwenden wir das [digitale Höhenmodell von NASA/CGIAR](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4)

In [3]:
# das Höhenmodell
srtm = ee.Image('CGIAR/SRTM90_V4')

### Beispiel: Wert für einen Punkt bestimmen

Oftmals wollen wir die Werte eines Rasters für einen bestimmten Punkt bestimmen:

In [4]:
# Kalrsuhe ...
point = ee.Geometry.Point(8.4, 49.0)

# extract the image value
data = srtm.select("elevation") \
    .reduceRegion(ee.Reducer.first(), point, 10) \
    .get("elevation")

data.getInfo()

118

### Beispiel: Gefälle berechnen

Hier wird die Funktion *slope* verwendet, welche bereits eingebaut ist

In [8]:
# Berechnung des Gefälles
slope = ee.Terrain.slope(srtm)

# Ergebnis anzeigen ...
globe = geemap.Map()
globe.setCenter(6.2, 46.210, 8)
vis = {'min': 0, 'max': 60, 'palette': cm.palettes.jet}
globe.addLayer(slope, vis, 'slope', opacity=0.7)
globe

Map(center=[46.21, 6.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Trick. Eine Region auf der Karte auswählen:

In [ ]:
select = globe.draw_last_feature.geometry().getInfo()
select

### Beispiel: Statistik des Gefälles

In [ ]:
# Median des Gefälles berechnen
stats = slope.reduceRegion(
    reducer = ee.Reducer.median(),
    geometry = select,
    scale = 30,
    maxPixels = 1e9)

stats.getInfo()

Eine Kombination:

In [ ]:
# a combination of reducers
red_comb = ee.Reducer.mean().combine(
  reducer2 = ee.Reducer.stdDev(),
  sharedInputs = True
).combine(
  reducer2 = ee.Reducer.min(),
  sharedInputs = True
).combine(
  reducer2 = ee.Reducer.max(),
  sharedInputs = True
)

# Min, max and standard deviation
stats = slope.reduceRegion(
    reducer = red_comb,
    geometry = select,
    scale = 30,
    maxPixels = 1e9)

stats.getInfo()

## Datensätze exportieren

Eine generelle Lösung für den Export von Datensätzen ist es, diese gezielt zum Download vorzubereiten:

In [ ]:
# Get a download URL for an image
image = ee.Image('srtm90_v4')

path = image.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': [[-120, 35], [-119, 35], [-119, 34], [-120, 34]]
})

print(path)

## *ee.FeatureCollections* inspizieren

Zuerst laden wir die Ländergrenzen aus einem *ee.FeatureCollection* des [US Department of State, Office of Geography](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017):

In [ ]:
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

Schauen wir uns die Metadaten eines Elements der Kollektion an:

In [ ]:
countries.first().getInfo()

Eine Liste der Länder erstellen:

In [ ]:
contry_list = countries.aggregate_array('country_na')
contry_list.getInfo()[:20]

Wo ist Deutschland? Finden wir das Polygon:

In [ ]:
germany = countries.filter(ee.Filter.eq('country_na', 'Germany'))
germany.getInfo()

Holen wir uns die WGS84 Koordinaten des Zentrums als Information:

In [ ]:
germany.geometry().centroid().getInfo()['coordinates']

Schauen wir uns das Polygon in einer Karte an:

In [ ]:
globe = geemap.Map(location=[49.014, 8.405], zoom_start=4)

vis_params = {'color': "red", 'width': 0.5}
globe.addLayer(germany, vis_params, name='Germany')
globe.centerObject(germany)

globe

## *ee.Feature* exportieren

Das Polygon als Shapefile exportieren:

In [ ]:
geemap.ee_export_vector(germany, 'data/germany.shp')

## *ee.Image* Regionen exportieren

In [ ]:
globe = geemap.Map(location=[49.014, 8.405], zoom=10)

image = ee.Image('USGS/SRTMGL1_003')

vis_params = {'min': 100, 'max': 800, 'palette': cm.palettes.dem}
globe.addLayer(image, vis_params, 'SRTM DEM', True, 0.9)

KA = ee.Geometry.Point([8.407, 49.015])
globe.addLayer(KA)

globe

### Regional begrenztes Gebiet exportieren

In [ ]:
roi = ee.Geometry.Rectangle([8.1, 48.9, 8.7, 49.1])
image_clipped = image.clip(roi)

geemap.ee_export_image(image_clipped, "data/DEM_Karlsruhe.tif", region=roi, file_per_band=True)

### Gebiet in der Karte auswählen

In [ ]:
polygon = globe.draw_last_feature.geometry()

geemap.ee_export_image(image, "data/DEM_selected.tif", region=polygon, file_per_band=True)

## *ee.ImageCollections* inspizieren

Hier verwenden wir stündliche Klimareanalysedaten [ERA5 des European Centre for Medium-Range Weather Forecasts](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY). Zuerst schauen wir uns die Dateneigenschaften an:

In [ ]:
collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
collection.first().bandNames().getInfo()

Was für Eigenschaften hat die Zeitreihe:

In [ ]:
dates = collection.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list')
datelist = pd.to_datetime(dates.getInfo(), unit='ms')
datelist

## *ee.ImageCollections* exportieren

Jetzt können wir Daten aussuchen und exportieren:

In [ ]:
roi = ee.Geometry.Rectangle([8.1, 48.9, 8.7, 49.1])

collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterBounds(roi) \
    .filterDate('2021-03-31T00:00:00', '2021-03-31T23:00:00') \
    .filter(ee.Filter.listContains("system:band_names", "temperature_2m"))

geemap.ee_export_image_collection(collection, region=roi, out_dir='data')

## Zeitreihen in *Pandas* exportieren

Daten aus *ee.ImageCollections* können auch als Zeitreihe extrahiert werden:

In [ ]:
KA = ee.Geometry.Point(8.405, 49.014)
buffer = 100

land_temp = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterDate('2021-01-01T00:00:00', '2021-03-31T00:00:00')

Schauen wir uns einen beispielhaften Datenpunkt an:

In [ ]:
# Calculate and print the mean value of the LST collection at the point
land_temp_point = land_temp.mean().sample(KA, buffer).first().get('temperature_2m').getInfo()
print('Average for KA at 1 Jan 2021 00:00:', round(land_temp_point - 273.15, 2), '°C')

Laden wir die Zeitreihe für Karlsruhe herunter (das kann eine Weile dauern):

In [ ]:
# Get the data for the point in urban area.
land_temp_poi = land_temp.getRegion(KA, buffer).getInfo()
len(land_temp_poi)

Wir definieren eine Funktion zur Speicherung der Daten als *pandas.DataFrame*:

In [ ]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

Nun können wir die gewünschten Daten extrahieren:

In [ ]:
land_temp_df = ee_array_to_df(land_temp_poi, ['temperature_2m'])

# konvertiere in Celsius
land_temp_df['temperature_2m'] -= 273.15

# als CSV speichern
land_temp_df.to_csv("data/KA_temperature-2m.csv")

land_temp_df

## ENDE